# Requirements

In [1]:
import sddk
import json
import re
import pandas as pd
import nltk
pd.options.display.max_columns = 1000 # to see all columns
import warnings
warnings.filterwarnings('ignore')

# Loading datasets

In [2]:
EDHCS = sddk.read_file("EDHCSg.geojson", "gdf", "a9237c5ea642d4714bcdefb03f70a1f4")

reading file located in a public folder


# Function to extract occupations

In [3]:
deities_csv = pd.read_csv("../data/deities_decline.csv").reset_index()
deities_csv

,index,nom_sg,gen_sg,dat_sg,acc_sg,abl_sg,voc_sg,special_suffix1,special_suffix2,special_suffix3
0,Asclepius,Asclepii,Asclepio,Asclepium,Asclepio,Asclepie,NaN,NaN,NaN,NaN
1,Aesculapius,Aesculapii,Aesculapio,Aesculapium,Aesculapio,Aesculapie,Aesculapi,Aesculapem,Aesculapeo,NaN
2,Esculapius,Esculapii,Esculapio,Esculapium,Esculapio,Esculapie,NaN,NaN,NaN,NaN
3,Asclepios,Asclepii,Asclepio,Asclepium,Asclepio,Asclepie,NaN,NaN,NaN,NaN
4,Apollo,Apollonis,Apolloni,Apollonem,Apollone,Apollo,NaN,NaN,NaN,NaN
5,Apollon,Apollonis,Apolloni,Apollonem,Apollone,Apollon,NaN,NaN,NaN,NaN
6,Apolon,Apolonis,Apoloni,Apolonem,Apolone,Apolon,NaN,NaN,NaN,NaN
7,Apolin,Apolinis,Apolini,Apolinem,Apoline,Apolin,NaN,NaN,NaN,NaN
8,Apollin,Apollinis,Apollini,Apollinem,Apolline,Apollin,NaN,NaN,NaN,NaN
9,Iupiter Optimus Maximus,Iovis Optimi Maximi,Iovi Optimo Maximo,Iovem Optimum Maximum,Iove Optimo Maximo,Iupiter Optimus Maximus,NaN,NaN,NaN,NaN


In [4]:
deities_dict = {}

asclep = []
for n in range(4):
    asclep.extend([el for el in deities_csv.loc[n].tolist() if isinstance(el, str)])
deities_dict["asclep"] = list(set(asclep))

apollo = []
for n in range(4,9):
    apollo.extend([el for el in deities_csv.loc[n].tolist() if isinstance(el, str)])
deities_dict["apollo"] = list(set(apollo))

jupiter = []
for n in range(9,13):
    jupiter.extend([el for el in deities_csv.loc[n].tolist() if isinstance(el, str)])
deities_dict["jupiter"] = list(set(jupiter))

In [5]:
deities_dict["apollo"]

['Apollone',
 'Apolonem',
 'Apolloni',
 'Apollini',
 'Apolon',
 'Apolline',
 'Apollonem',
 'Apolonis',
 'Apolone',
 'Apollinem',
 'Apollinis',
 'Apollon',
 'Apoloni',
 'Apoline',
 'Apolinem',
 'Apolinis',
 'Apolin',
 'Apollo',
 'Apollonis',
 'Apollin',
 'Apolini']

In [24]:
def eval_list(people_list):
    try: return eval(people_list)
    except: return []
EDHCS["people"] = EDHCS["people"].apply(eval_list)

In [7]:
def test_extract(text):
    try:
        if re.search("(\W|^)Apollini(\W|$)", text):
            return True
        else:
            return False
    except:
        return False
len(EDHCS[EDHCS["clean_text_interpretive_word"].apply(test_extract)])

361

In [8]:
def extract_deity(inscription_text, deity):
    if not isinstance(inscription_text, str): # if not valid string
        inscription_text = ""
    to_return = False
    for deity_morph in deities_dict[deity]:
        if deity_morph in inscription_text: # first check it this way, otherwise skip
            if re.search("(\W|^)" + deity_morph + "(\W|$)", inscription_text):
                to_return = True
                break
    return to_return

In [9]:
EDHCS["asclepius"] = EDHCS["clean_text_interpretive_word"].apply(extract_deity, deity="asclep")
EDHCS["apollo"] = EDHCS["clean_text_interpretive_word"].apply(extract_deity, deity="apollo")
EDHCS["jupiter"] = EDHCS["clean_text_interpretive_word"].apply(extract_deity, deity="jupiter")

In [10]:
print(len(EDHCS[EDHCS["asclepius"]]))
print(len(EDHCS[EDHCS["apollo"]]))
print(len(EDHCS[EDHCS["jupiter"]]))

238
495
2868


In [ ]:
# however, we have to check the people attribute...

def check_name_in_people(people_list, name):
    try: return any([el for el in people_list if re.search("(\W|^){0}(\W|$)".format(name), el["nomen"]) or re.search("(\W|^){0}(\W|$)".format(name), el["cognomen"])])
    except: False

name = "Asclepius"
EDHCS["asclepius_people"] = EDHCS["people"].apply(lambda x: check_name_in_people(x, name))
name = "Apollo"
EDHCS["apollo_people"] = EDHCS["people"].apply(lambda x: check_name_in_people(x, name))

In [48]:
def check_people(deity , people_deity):
    if deity: 
        if people_deity:
            return False
        else:
            return True
    else:
        return False

EDHCS["asclepius"] = EDHCS.apply(lambda row: check_people(row["asclepius"], row["asclepius_people"]), axis=1)
EDHCS["apollo"] = EDHCS.apply(lambda row: check_people(row["apollo"], row["apollo_people"]), axis=1)

In [47]:
len(EDHCS[EDHCS["asclepius"]])

230

In [49]:
len(EDHCS[EDHCS["apollo"]])

491

In [50]:
EDHCS.drop(["asclepius_people", "apollo_people"], axis=1, inplace=True)

ok, it is not bad, previously we had 156, 306, and 2598 in EDH and 90, 205, and 391 in EDCS.

In [51]:
EDHCS.to_file("../data/large_files/EDHCS_deities.geojson", driver="GeoJSON")

ValueError: Invalid field type <class 'list'>